In [9]:
import imaplib
import email
from email.header import decode_header
import os
import pandas as pd
import re

# Datos del usuario
username = "csoto981217@gmail.com"
password = "xtwm pmeh etfv dfda"

# Crear conexión
imap = imaplib.IMAP4_SSL("imap.gmail.com")
# Iniciar sesión
imap.login(username, password)

# Seleccionar la bandeja de entrada (INBOX)
imap.select("INBOX")

# Buscar correos no leídos
status, mensajes = imap.search(None, 'UNSEEN')

# mensajes a recibir (solo los últimos 15 no leídos)
N = 15
mensajes_ids = mensajes[0].split()
if len(mensajes_ids) >= N:
    mensajes_no_leidos = mensajes_ids[-N:]  # Obtener los últimos N correos no leídos
else:
    mensajes_no_leidos = mensajes_ids  # Obtener todos los correos no leídos si hay menos de N

# Lista para almacenar los datos de los correos
data = []

for num in mensajes_no_leidos:
    try:
        res, mensaje = imap.fetch(num, "(RFC822)")
    except:
        break
    
    for respuesta in mensaje:
        if isinstance(respuesta, tuple):
            # Obtener el contenido del correo
            mensaje = email.message_from_bytes(respuesta[1])
            # Decodificar el asunto del correo
            subject, encoding = decode_header(mensaje["Subject"])[0]
            if isinstance(subject, bytes):
                # Convertir a string
                subject = subject.decode(encoding if encoding else 'utf-8', errors='replace')
            # De donde viene el correo
            from_ = mensaje.get("From")
            
            # Inicializar el cuerpo del correo
            body = ""

            if mensaje.is_multipart():
                # Recorrer las partes del correo
                for part in mensaje.walk():
                    # Extraer el contenido
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # El cuerpo del correo
                        part_body = part.get_payload(decode=True).decode('utf-8', errors='replace')
                    except:
                        part_body = ""
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # Concatenar el cuerpo del correo
                        body += part_body
            else:
                # Contenido del mensaje
                content_type = mensaje.get_content_type()
                # Cuerpo del mensaje
                body = mensaje.get_payload(decode=True).decode('utf-8', errors='replace')
            
            # Añadir los datos del correo a la lista
            data.append([subject, from_, body])
            print("Mensaje obtenido con éxito")

            # Marcar el correo como leído
            imap.store(num, '+FLAGS', '\\Seen')

# Cerrar la conexión
imap.close()
imap.logout()

# Crear el DataFrame
df = pd.DataFrame(data, columns=["Subject", "From", "Mensaje"])

# Función para limpiar el contenido del mensaje
def limpiar_mensaje(mensaje):
    # Eliminar espacios en blanco al inicio y al final
    mensaje = mensaje.strip()
    # Eliminar caracteres especiales usando expresiones regulares
    mensaje = re.sub(r'[^\w\s]', '', mensaje)
    # Eliminar múltiples espacios y convertirlos a un solo espacio
    mensaje = re.sub(r'\s+', ' ', mensaje)
    # Convertir a minúsculas
    mensaje = mensaje.lower()
    return mensaje

# Aplicar la función de limpieza a la columna 'Mensaje'
df['Mensaje'] = df['Mensaje'].apply(limpiar_mensaje)

# Mostrar el DataFrame con los mensajes limpios
df


Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito
Mensaje obtenido con éxito


,Subject,From,Mensaje
0,HOY: Asamblea de Juventudes. ¡Ven a la sede!,Colombia Humana <boletines@colombiahumana.co>,imagehttpscrmcolombiahumanacowpcontentuploads2...
1,Alerta de seguridad,Google <no-reply@accounts.google.com>,image google un nuevo acceso en el dispositivo...
2,Camilo ¡Line up Platzi Conf confirmado! + 30 s...,"""Ana de Platzi"" <team@t.platzi.com>",los cupos se agotan asegura tu lugar httpsplat...
3,¡Tenemos un nuevo taller gratuito para ti!,=?UTF-8?Q?Estefi_de_Desaf=C3=ADo_Latam?= <insc...,asegura un cupo para este taller gratuito del ...
4,“Camilo Soto”: 1 mención en un artículo sobre ...,Mentioned by Julio Cajamarca Villa <premium@ac...,camilo el nombre camilo soto fue mencionado en...
5,TaDa Delivery: Encuesta de,TaDa Delivery <colombia@contacto.tadadelivery....,encuesta de satisfacción hola nos encantaría ...
6,Razones por las que las empresas no…,LinkedIn <notifications-noreply@linkedin.com>,razones por las que las empresas no contratan ...
7,Vamos por otro paso hoy 🔥,=?UTF-8?Q?Academia_Desaf=C3=ADo_Latam?=\r\n <i...,el mundo necesita de tu conocimiento ya hicist...
8,"📄 ""Book-Review-Dynamical Systems-III"" by musa ...",PDF By musa masing <updates@academia-mail.com>,you read the paper a hrefhttpswwwacademiaeduke...
9,¡Tienes un descuento! ¿Viajamos?,Uber <uber@uber.com>,doctype html html xmlnshttpwwww3org1999xhtml x...


In [10]:
df

,Subject,From,Mensaje
0,HOY: Asamblea de Juventudes. ¡Ven a la sede!,Colombia Humana <boletines@colombiahumana.co>,imagehttpscrmcolombiahumanacowpcontentuploads2...
1,Alerta de seguridad,Google <no-reply@accounts.google.com>,image google un nuevo acceso en el dispositivo...
2,Camilo ¡Line up Platzi Conf confirmado! + 30 s...,"""Ana de Platzi"" <team@t.platzi.com>",los cupos se agotan asegura tu lugar httpsplat...
3,¡Tenemos un nuevo taller gratuito para ti!,=?UTF-8?Q?Estefi_de_Desaf=C3=ADo_Latam?= <insc...,asegura un cupo para este taller gratuito del ...
4,“Camilo Soto”: 1 mención en un artículo sobre ...,Mentioned by Julio Cajamarca Villa <premium@ac...,camilo el nombre camilo soto fue mencionado en...
5,TaDa Delivery: Encuesta de,TaDa Delivery <colombia@contacto.tadadelivery....,encuesta de satisfacción hola nos encantaría ...
6,Razones por las que las empresas no…,LinkedIn <notifications-noreply@linkedin.com>,razones por las que las empresas no contratan ...
7,Vamos por otro paso hoy 🔥,=?UTF-8?Q?Academia_Desaf=C3=ADo_Latam?=\r\n <i...,el mundo necesita de tu conocimiento ya hicist...
8,"📄 ""Book-Review-Dynamical Systems-III"" by musa ...",PDF By musa masing <updates@academia-mail.com>,you read the paper a hrefhttpswwwacademiaeduke...
9,¡Tienes un descuento! ¿Viajamos?,Uber <uber@uber.com>,doctype html html xmlnshttpwwww3org1999xhtml x...


In [11]:
df.to_excel('mensajes_clasificados.xlsx',index=False)

In [13]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from transformers import GPT2Tokenizer, GPTNeoForSequenceClassification, pipeline
from getpass import getpass

# Crear un pipeline de análisis de sentimiento usando GPT-Neo
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
model = GPTNeoForSequenceClassification.from_pretrained("EleutherAI/gpt-neo-2.7B", num_labels=2)

# Usamos un pipeline de Hugging Face para hacer la clasificación de sentimientos
pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Función para truncar mensajes largos
def truncar_mensaje(mensaje, max_length=512):
    tokens = tokenizer.encode(mensaje, add_special_tokens=True)
    if len(tokens) > max_length:
        tokens = tokens[:max_length-1] + [tokenizer.sep_token_id]
    return tokenizer.decode(tokens, skip_special_tokens=True)

# Función para aplicar el modelo y obtener la predicción
def obtener_prediccion(mensaje):
    mensaje_truncado = truncar_mensaje(mensaje)
    resultado = pipe(mensaje_truncado)[0]  # Obtenemos el primer resultado
    return resultado['label'], resultado['score']  # Devolvemos la etiqueta y la puntuación

# Función para enviar un correo electrónico
def enviar_correo(subject, body, to_email, from_email, password):
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(from_email, password)
        text = msg.as_string()
        server.sendmail(from_email, to_email, text)
        server.quit()
        print("Correo enviado exitosamente a", to_email)
    except Exception as e:
        print(f"Error al enviar el correo a {to_email}: {e}")

# Credenciales de correo electrónico
from_email = username
password = password
negative_email = "csoto981217@gmail.com"
positive_email = "csoto981217@gmail.com"

# Aplicar la función a cada mensaje en el DataFrame
def analizar_y_notificar(row):
    try:
        label, score = obtener_prediccion(row['Mensaje'])
        if label == 'NEGATIVE':
            subject = "Alerta de Mensaje negativo"
            body = f"Se ha detectado un Mensaje negativo:\n\n{row['Mensaje']}\n\nConfianza: {score:.2f}"
            enviar_correo(subject, body, negative_email, from_email, password)
        elif label == 'POSITIVE':
            subject = "Alerta de mensaje positivo"
            body = f"Se ha detectado un Mensaje positivo:\n\n{row['Mensaje']}\n\nConfianza: {score:.2f}"
            enviar_correo(subject, body, positive_email, from_email, password)
        return label, score
    except Exception as e:
        print(f"Error al procesar el mensaje: {e}")
        return None, None

# Ejemplo de DataFrame
#data = {'Mensaje': ["I'm happy with the service!", "This is terrible and I'm very upset."]}
#df = pd.DataFrame(data)

# Aplicar la función al DataFrame
df[['prediccion', 'confianza']] = df.apply(analizar_y_notificar, axis=1, result_type="expand")

# Mostrar el DataFrame con las predicciones
print(df)


ModuleNotFoundError: No module named 'transformers'

In [ ]:
df

,Subject,From,Mensaje,prediccion,confianza
0,Las Grandes Pantallas se toman Ktronix 📣,"""Ktronix"" <boletin@email.ktronix.com>",httpsclickemailktronixcomqs5cdc536a569f15b871d...,None,NaN
1,¡Únete,Transkriptor <no-reply@transkriptor.com>,doctype html public w3cdtd xhtml 10 transition...,None,NaN
2,"Estudia y trabaja en España, ¡últimos cupos!",=?UTF-8?Q?Tu_carrera_en_Espa=C3=B1a?= <publici...,html head titleemail espantildeatitle meta htt...,None,NaN
3,😃☀️ ¡Vive el Hotel los Puentes! Más cerca de l...,Comfacundi <comunicados@comfacundi.com.co>,para visualizar correctamente este correo elec...,LABEL_1,0.939928
4,Asiste a la cuarta sesión de la Cátedra Utadeo...,Oficina de Egresados Utadeo <oficina.egresados...,newsletter nbsp oficina de egresados pbx 242 7...,LABEL_1,0.994775
5,"nomegustapeinarme, sebasbuitragoa_ y leonardo....","""Instagram"" <no-reply@mail.instagram.com>",doctype html public w3cdtd html 401 transition...,None,NaN
6,Es tu carrera,"""edX Team"" <noreply@sfmc.edx.org>",httpsclicksfmcedxorgqs14cd28d13df558b473021fef...,None,NaN
7,Asegúrate una victoria deliciosa con un 𝟐𝟓% 𝐝𝐞...,=?ISO-8859-1?Q?McDonald=B4s_App?=\r\n <McDonal...,httpsd2eayappgoogllinkhttpswwwmcdonaldscomcoco...,LABEL_1,0.980203
8,Llegó tu Factura Claro Móvil,facturasclarocol@claro.com.co,,None,NaN
9,Descubre España: Estudia y trabaja hasta 30 ho...,=?UTF-8?Q?Estudios_en_Espa=C3=B1a_-_Tu_puerta_...,html head titleemail espanatitle meta httpequi...,None,NaN
